# Introduction to `pyspark.sql.DataFrame`s

Adapted from [Databrick's tutorial](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html)

In [1]:
# import pyspark class Row from module sql
from pyspark.sql import *

## What is spark?

* Build for the Hadoop platform
* Replacement of MapReduce
* Second-generation optimization
    * Lazy
    * Optimized
    * Persistent data structures
* Written in scala

## Ok ... so what's Hadoop?

* Distributed computing platform
* [Used by lots of companies](https://wiki.apache.org/hadoop/PoweredBy)
* Becoming an industry standard


## What is `pyspark`?

* Python interface to spark
* Needs a spark session
    * `session` $\leftrightarrow$ spark


## Step 0 - Create a spark session

* `pyspark` communicates with `spark` through a session
* Similar to `sqlalchemy` session.

In [2]:
spark = SparkSession.builder.appName('Ops').getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

22/10/28 23:04:37 WARN Utils: Your hostname, kg3597wc201 resolves to a loopback address: 127.0.1.1; using 192.168.1.248 instead (on interface wifi0)
22/10/28 23:04:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/28 23:04:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Overview -  `pyspark.DataFrame`

* A `DataFrame` is a collection of `Row`s
* `Row`s can be distributed over many machines
* `spark`
    * Hides the messy details
    * Optimizes operations

## Creating a `Row` of data

* Use the `Row` class
* Pass data using keywords
    * key == column name
    * value == cell value

In [3]:
department1 = Row(id='123456', name='Computer Science')
department1

Row(id='123456', name='Computer Science')

## Unpacking a `Row` dictionary

* Data is in a row dictionary
* Unpack keywords using `**`

In [4]:
dept2_info = {'id':'789012', 'name':'Mechanical Engineering'}
department2 = Row(**dept2_info)
department2

Row(id='789012', name='Mechanical Engineering')

## Unpacking a list of row dictionaries

In [5]:
dept_info = [{'id':123456, 'name':'Computer Science'},
             {'id':789012, 'name':'Mechanical Engineering'},
             {'id':345678, 'name':'Theater and Drama'},
             {'id':901234, 'name':'Indoor Recreation'}]

dept_rows = [Row(**r) for r in dept_info]
dept_rows

[Row(id=123456, name='Computer Science'),
 Row(id=789012, name='Mechanical Engineering'),
 Row(id=345678, name='Theater and Drama'),
 Row(id=901234, name='Indoor Recreation')]

## Access `Row` content with column attributes

In [6]:
[dept.id for dept in dept_rows]

[123456, 789012, 345678, 901234]

In [7]:
[dept.name for dept in dept_rows]

['Computer Science',
 'Mechanical Engineering',
 'Theater and Drama',
 'Indoor Recreation']

## Creating a `pyspark.DataFrame`

* A `DataFrame` is a collection of `Row`s
* Create with spark.createDataFrame
* Need to have a 

In [8]:
df = spark.createDataFrame(dept_rows)
df

DataFrame[id: bigint, name: string]

## How to think about a `pyspark.DataFrame`

<img src="./img/pyspark_df.png?raw=1" width=600>

## Example - `filter` and `collect`

In [9]:
output = (df
            .filter(df.name.startswith('C'))
            .collect())
output

[Row(id=123456, name='Computer Science')]

## Why is `pyspark` so slow

* Optimized for 
    * Distributed computation
    * Big data 
* Not great for
    * Local work
    * Small data

## `filter` and `collect` illustrated

<img src="./img/pyspark_filter_collect.gif" width=600>

## Reading a `csv` file with `spark.read.csv`

In [10]:
heroes = spark.read.csv('./data/heroes_information.csv', header=True)
heroes

DataFrame[_c0: string, name: string, Gender: string, Eye color: string, Race: string, Hair color: string, Height: string, Publisher: string, Skin color: string, Alignment: string, Weight: string]

## Inspecting the column types

In [11]:
heroes.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Eye color: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Hair color: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Skin color: string (nullable = true)
 |-- Alignment: string (nullable = true)
 |-- Weight: string (nullable = true)



## Gathering results in `pyspark.sql`

* **Important fact** All `pyspark` queries end in a collection method
* **Why?** Data is (possibly) spread across many machines
* <font color = "red"> **Warning** This might be is *expensive*! Know how much data your are requesting! </font>

## Gathering methods

* `collect` returns all values
* `take(n)` returns the first `n` values 
* `sample(n)` returns `n` randomly selected values 

## Inspecting the content - `take`

In [12]:
heroes.take(5)

[Row(_c0='0', name='A-Bomb', Gender='Male', Eye color='yellow', Race='Human', Hair color='No Hair', Height='203.0', Publisher='Marvel Comics', Skin color='-', Alignment='good', Weight='441.0'),
 Row(_c0='1', name='Abe Sapien', Gender='Male', Eye color='blue', Race='Icthyo Sapien', Hair color='No Hair', Height='191.0', Publisher='Dark Horse Comics', Skin color='blue', Alignment='good', Weight='65.0'),
 Row(_c0='2', name='Abin Sur', Gender='Male', Eye color='blue', Race='Ungaran', Hair color='No Hair', Height='185.0', Publisher='DC Comics', Skin color='red', Alignment='good', Weight='90.0'),
 Row(_c0='3', name='Abomination', Gender='Male', Eye color='green', Race='Human / Radiation', Hair color='No Hair', Height='203.0', Publisher='Marvel Comics', Skin color='-', Alignment='bad', Weight='441.0'),
 Row(_c0='4', name='Abraxas', Gender='Male', Eye color='blue', Race='Cosmic Entity', Hair color='Black', Height='-99.0', Publisher='Marvel Comics', Skin color='-', Alignment='bad', Weight='-99.0

## Inspecting the content - `sample`

In [13]:
sample = heroes.sample(fraction=0.01).collect()
sample

[Row(_c0='32', name='Anti-Spawn', Gender='Male', Eye color='-', Race='-', Hair color='-', Height='-99.0', Publisher='Image Comics', Skin color='-', Alignment='bad', Weight='-99.0'),
 Row(_c0='81', name='Big Daddy', Gender='Male', Eye color='-', Race='-', Hair color='-', Height='-99.0', Publisher='Icon Comics', Skin color='-', Alignment='good', Weight='-99.0'),
 Row(_c0='175', name='Chuck Norris', Gender='Male', Eye color='-', Race='-', Hair color='-', Height='178.0', Publisher=None, Skin color='-', Alignment='good', Weight='-99.0'),
 Row(_c0='228', name='Donna Troy', Gender='Female', Eye color='blue', Race='Amazon', Hair color='Black', Height='175.0', Publisher='DC Comics', Skin color='-', Alignment='good', Weight='63.0'),
 Row(_c0='239', name='Elongated Man', Gender='Male', Eye color='blue', Race='-', Hair color='Red', Height='185.0', Publisher='DC Comics', Skin color='-', Alignment='good', Weight='80.0'),
 Row(_c0='371', name='Jubilee', Gender='Female', Eye color='red', Race='Mutant'

## Switching to a `pd.DataFrame` (because that was UGLY)

You can pipe the results into `more_pyspark.to_pandas` to get the results in a dataframe

In [14]:
from more_pyspark import to_pandas

sample >> to_pandas

,_c0,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,32,Anti-Spawn,Male,-,-,-,-99.0,Image Comics,-,bad,-99.0
1,81,Big Daddy,Male,-,-,-,-99.0,Icon Comics,-,good,-99.0
2,175,Chuck Norris,Male,-,-,-,178.0,None,-,good,-99.0
3,228,Donna Troy,Female,blue,Amazon,Black,175.0,DC Comics,-,good,63.0
4,239,Elongated Man,Male,blue,-,Red,185.0,DC Comics,-,good,80.0
5,371,Jubilee,Female,red,Mutant,Black,165.0,Marvel Comics,-,good,52.0
6,478,Multiple Man,Male,blue,-,Brown,180.0,Marvel Comics,-,good,70.0
7,614,Spawn,Male,brown,Demon,Black,211.0,Image Comics,-,good,405.0
8,699,Vision,Male,gold,Android,No Hair,191.0,Marvel Comics,red,good,135.0


## Getting all results with `collect`

<font color = "red"> **Warning** This might be is *expensive*! Know how much data your are requesting! </font> 

**The `collect` rule:** `count` before `collect`

In [15]:
heroes.collect() >> to_pandas # <-- probably don't do this

,_c0,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441.0
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441.0
4,4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0
...,...,...,...,...,...,...,...,...,...,...,...
729,729,Yellowjacket II,Female,blue,Human,Strawberry Blond,165.0,Marvel Comics,-,good,52.0
730,730,Ymir,Male,white,Frost Giant,No Hair,304.8,Marvel Comics,white,good,-99.0
731,731,Yoda,Male,brown,Yoda's species,White,66.0,George Lucas,green,good,17.0
732,732,Zatanna,Female,blue,Human,Black,170.0,DC Comics,-,good,57.0


In [16]:
heroes.filter(heroes['Eye Color'] == 'blue').collect() >> to_pandas # <-- better but still might be lots, let's check

,_c0,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
1,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
2,4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0
3,5,Absorbing Man,Male,blue,Human,No Hair,193.0,Marvel Comics,-,bad,122.0
4,6,Adam Monroe,Male,blue,-,Blond,-99.0,NBC - Heroes,-,good,-99.0
...,...,...,...,...,...,...,...,...,...,...,...
220,726,X-Man,Male,blue,-,Brown,175.0,Marvel Comics,-,good,61.0
221,727,Yellow Claw,Male,blue,-,No Hair,188.0,Marvel Comics,-,bad,95.0
222,728,Yellowjacket,Male,blue,Human,Blond,183.0,Marvel Comics,-,good,83.0
223,729,Yellowjacket II,Female,blue,Human,Strawberry Blond,165.0,Marvel Comics,-,good,52.0


In [17]:
# Iverson's Law -- Count before you collect!
heroes.filter(heroes['Eye Color'] == 'blue').count()

225

## Did you notice?

<img src="./img/pyspark_missing_values.png?raw=1" width=400>

## Specifying a `nullValue`

In [18]:
heroes = spark.read.csv('./data/heroes_information.csv', header=True, nullValue='-')
heroes.take(5) >> to_pandas

,_c0,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,None,good,441.0
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,None,bad,441.0
4,4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,None,bad,-99.0


## Did you notice?

<img src="./img/pyspark_default_types.png?raw=1" width=400>

Default type is a string

## Letting `spark` guess the types

Set `inferScheme=True` 

In [19]:
heroes = spark.read.csv('./data/heroes_information.csv', header=True, inferSchema=True, nullValue='-')
heroes

DataFrame[_c0: int, name: string, Gender: string, Eye color: string, Race: string, Hair color: string, Height: double, Publisher: string, Skin color: string, Alignment: string, Weight: double]

## Checking the column types after `inferScheme`

In this case, `spark` guessed correctly

In [20]:
heroes.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Eye color: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Hair color: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Skin color: string (nullable = true)
 |-- Alignment: string (nullable = true)
 |-- Weight: double (nullable = true)



## Inspecting the content - `take`

In [21]:
heroes.take(5)

[Row(_c0=0, name='A-Bomb', Gender='Male', Eye color='yellow', Race='Human', Hair color='No Hair', Height=203.0, Publisher='Marvel Comics', Skin color=None, Alignment='good', Weight=441.0),
 Row(_c0=1, name='Abe Sapien', Gender='Male', Eye color='blue', Race='Icthyo Sapien', Hair color='No Hair', Height=191.0, Publisher='Dark Horse Comics', Skin color='blue', Alignment='good', Weight=65.0),
 Row(_c0=2, name='Abin Sur', Gender='Male', Eye color='blue', Race='Ungaran', Hair color='No Hair', Height=185.0, Publisher='DC Comics', Skin color='red', Alignment='good', Weight=90.0),
 Row(_c0=3, name='Abomination', Gender='Male', Eye color='green', Race='Human / Radiation', Hair color='No Hair', Height=203.0, Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight=441.0),
 Row(_c0=4, name='Abraxas', Gender='Male', Eye color='blue', Race='Cosmic Entity', Hair color='Black', Height=-99.0, Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight=-99.0)]

## Explicit `schema` specification

Format is `add(name, type, nullable?)`

In [22]:
from pyspark.sql.types import StructType
from pyspark.sql.types import DoubleType, StringType, IntegerType

hero_schema = (StructType()
  .add('Id', IntegerType(), False)
  .add('name', StringType(), True)
  .add('Gender', StringType(), True)
  .add('Eye color', StringType(), True)
  .add('Race', StringType(), True)
  .add('Hair color', StringType(), True)
  .add('Height', DoubleType(), True)
  .add('Publisher', StringType(), True)
  .add('Skin color', StringType(), True)
  .add('Alignment', StringType(), True)
  .add('Weight', DoubleType(), True))

heroes = spark.read.csv('./data/heroes_information.csv', header=True, schema=hero_schema, nullValue='-')
heroes

DataFrame[Id: int, name: string, Gender: string, Eye color: string, Race: string, Hair color: string, Height: double, Publisher: string, Skin color: string, Alignment: string, Weight: double]

In [23]:
heroes.take(5)

[Row(Id=0, name='A-Bomb', Gender='Male', Eye color='yellow', Race='Human', Hair color='No Hair', Height=203.0, Publisher='Marvel Comics', Skin color=None, Alignment='good', Weight=441.0),
 Row(Id=1, name='Abe Sapien', Gender='Male', Eye color='blue', Race='Icthyo Sapien', Hair color='No Hair', Height=191.0, Publisher='Dark Horse Comics', Skin color='blue', Alignment='good', Weight=65.0),
 Row(Id=2, name='Abin Sur', Gender='Male', Eye color='blue', Race='Ungaran', Hair color='No Hair', Height=185.0, Publisher='DC Comics', Skin color='red', Alignment='good', Weight=90.0),
 Row(Id=3, name='Abomination', Gender='Male', Eye color='green', Race='Human / Radiation', Hair color='No Hair', Height=203.0, Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight=441.0),
 Row(Id=4, name='Abraxas', Gender='Male', Eye color='blue', Race='Cosmic Entity', Hair color='Black', Height=-99.0, Publisher='Marvel Comics', Skin color=None, Alignment='bad', Weight=-99.0)]

## <font color="red"> Exercise 1 </font>

Define a `schema` and read in `./data/super_hero_powers.csv`

In [24]:
from pyspark.sql.types import StructType
from pyspark.sql.types import BooleanType, StringType 

# Your code here

In [25]:
# not manually typing all of that
nameglob = "hero_names,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,Danger Sense,Underwater breathing,Marksmanship,Weapons Master,Power Augmentation,Animal Attributes,Longevity,Intelligence,Super Strength,Cryokinesis,Telepathy,Energy Armor,Energy Blasts,Duplication,Size Changing,Density Control,Stamina,Astral Travel,Audio Control,Dexterity,Omnitrix,Super Speed,Possession,Animal Oriented Powers,Weapon-based Powers,Electrokinesis,Darkforce Manipulation,Death Touch,Teleportation,Enhanced Senses,Telekinesis,Energy Beams,Magic,Hyperkinesis,Jump,Clairvoyance,Dimensional Travel,Power Sense,Shapeshifting,Peak Human Condition,Immortality,Camouflage,Element Control,Phasing,Astral Projection,Electrical Transport,Fire Control,Projection,Summoning,Enhanced Memory,Reflexes,Invulnerability,Energy Constructs,Force Fields,Self-Sustenance,Anti-Gravity,Empathy,Power Nullifier,Radiation Control,Psionic Powers,Elasticity,Substance Secretion,Elemental Transmogrification,Technopath/Cyberpath,Photographic Reflexes,Seismic Power,Animation,Precognition,Mind Control,Fire Resistance,Power Absorption,Enhanced Hearing,Nova Force,Insanity,Hypnokinesis,Animal Control,Natural Armor,Intangibility,Enhanced Sight,Molecular Manipulation,Heat Generation,Adaptation,Gliding,Power Suit,Mind Blast,Probability Manipulation,Gravity Control,Regeneration,Light Control,Echolocation,Levitation,Toxin and Disease Control,Banish,Energy Manipulation,Heat Resistance,Natural Weapons,Time Travel,Enhanced Smell,Illusions,Thirstokinesis,Hair Manipulation,Illumination,Omnipotent,Cloaking,Changing Armor,Power Cosmic,Biokinesis,Water Control,Radiation Immunity,Vision - Telescopic,Toxin and Disease Resistance,Spatial Awareness,Energy Resistance,Telepathy Resistance,Molecular Combustion,Omnilingualism,Portal Creation,Magnetism,Mind Control Resistance,Plant Control,Sonar,Sonic Scream,Time Manipulation,Enhanced Touch,Magic Resistance,Invisibility,Sub-Mariner,Radiation Absorption,Intuitive aptitude,Vision - Microscopic,Melting,Wind Control,Super Breath,Wallcrawling,Vision - Night,Vision - Infrared,Grim Reaping,Matter Absorption,The Force,Resurrection,Terrakinesis,Vision - Heat,Vitakinesis,Radar Sense,Qwardian Power Ring,Weather Control,Vision - X-Ray,Vision - Thermal,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient"
colnames = nameglob.split(",")
len(colnames)

168

In [26]:
# you can add them one at a time, right?
some_schema = StructType()
some_schema.add("hero_names", StringType(), False)

StructType([StructField('hero_names', StringType(), False)])

In [27]:
# Right, that's good to know
hero_power_schema = StructType().add("hero_names", StringType(), False)

for col in colnames[1:]:
    hero_power_schema.add(col, BooleanType(), False)

In [28]:
hero_power_schema[-5:]

StructType([StructField('Phoenix Force', BooleanType(), False), StructField('Molecular Dissipation', BooleanType(), False), StructField('Vision - Cryo', BooleanType(), False), StructField('Omnipresent', BooleanType(), False), StructField('Omniscient', BooleanType(), False)])

In [29]:
hero_powers = spark.read.csv("./data/super_hero_powers.csv", header=True, schema=hero_power_schema)
hero_powers.take(1)

[Row(hero_names='3-D Man', Agility=True, Accelerated Healing=False, Lantern Power Ring=False, Dimensional Awareness=False, Cold Resistance=False, Durability=False, Stealth=False, Energy Absorption=False, Flight=False, Danger Sense=False, Underwater breathing=False, Marksmanship=False, Weapons Master=False, Power Augmentation=False, Animal Attributes=False, Longevity=False, Intelligence=False, Super Strength=True, Cryokinesis=False, Telepathy=False, Energy Armor=False, Energy Blasts=False, Duplication=False, Size Changing=False, Density Control=False, Stamina=True, Astral Travel=False, Audio Control=False, Dexterity=False, Omnitrix=False, Super Speed=True, Possession=False, Animal Oriented Powers=False, Weapon-based Powers=False, Electrokinesis=False, Darkforce Manipulation=False, Death Touch=False, Teleportation=False, Enhanced Senses=False, Telekinesis=False, Energy Beams=False, Magic=False, Hyperkinesis=False, Jump=False, Clairvoyance=False, Dimensional Travel=False, Power Sense=Fals

## `pyspark.sql` queries are like `SQL` queries

#### Filter, group, and aggregate (categorical)

In [30]:
(heroes
     .where(heroes.Gender == 'Male')
     .groupby(heroes['Eye color'])
     .count()
     .take(5)
) >> to_pandas

,Eye color,count
0,grey,6
1,green,30
2,yellow,16
3,bown,1
4,None,121


#### Group by multiple and aggregate (categorical)

In [31]:
(heroes
     .groupby(heroes['Eye color'], heroes.Gender)
     .count()
     .take(5)
) >> to_pandas

,Eye color,Gender,count
0,yellow (without irises),None,1
1,green,Male,30
2,violet,Female,2
3,hazel,Female,3
4,blue,Male,143


## <font color="red"> Exercise 2 </font>
    
Perform `pyspark.sql` queries to answer each of the following questions.

1. How many heroes have both Super Strength and Super Speed?
2. How many heroes have names that start with the word *Black*
3. Are heroes with Agility more likely to have Stealth?
4. What fraction of all heroes that can fly also have Super Strength?
5. Consider heroes that have names that contain `"girl"`, `"boy"`, `"woman"`, or `"man"`.  Compute the following ratio

$$\frac{N(\text{boy or man})}{N(\text{girl or woman)}}$$

**Hint:** You will need to use some combination of `where`, `group_by`, and `count` for each part.

In [32]:
# 1: How many heroes have both Super Strength and Super Speed?
(hero_powers
    .where(hero_powers["Super Strength"] 
               & hero_powers["Super Speed"])
    .count()
)

219

In [33]:
# 2: How many heroes have names that start with the word Black?
(hero_powers
    .where(hero_powers.hero_names.startswith("Black"))
    .count()
)

16

In [34]:
import pyspark.sql.functions as fn
from pyspark.sql.functions import col

In [35]:
# 3: Are heroes with Agility more likely to have Stealth [than heroes without agility]?
# agility & stealth / agility vs no agility & stealth / no agility

(hero_powers
    .groupby(hero_powers.Stealth, hero_powers.Agility)
    .count()
    .withColumn("percent_of_agility", fn.col("count") / fn.sum("count").over(Window.partitionBy("Agility")))
    .where(hero_powers.Stealth)
    .drop("Stealth")
    .collect()
) >> to_pandas

,Agility,count,percent_of_agility
0,False,31,0.072941
1,True,95,0.392562


In [36]:
# 4: What fraction of all heroes that can fly also have Super Strength?

(hero_powers
    .where(hero_powers.Flight)
    .groupby("Super Strength")
    .count()
    .withColumn("fraction", col("count")/fn.sum("count").over(Window.partitionBy()))
    .where(col("Super Strength"))
    .select("fraction")
    .collect()
) >> to_pandas

,fraction
0,0.693396


In [37]:
# 5: Consider heroes that have names that contain "girl", "boy", "woman", or "man". Compute the ratio
# assuming we don't actually want case-sensitivity
# use negative look-behind to avoid false positive of "woman" contains "man"

fem_pattern = "[gG]irl|[wW]oman"
masc_pattern = "[bB]oy|(?<![wW]o)[mM]an"

(hero_powers
    .where(col("hero_names").rlike(fem_pattern+"|"+masc_pattern))
    .select(fn.count(fn.when(col("hero_names").rlike(masc_pattern), True)).alias("male"),
            fn.count(fn.when(col("hero_names").rlike(fem_pattern), True)).alias("female"))
    .select("male", "female", (col("male") / col("female")).alias("ratio"))
    .collect()
) >> to_pandas

,male,female,ratio
0,54,26,2.076923


# Appendix

## Creating rows from list of data

## Creating a Row class

* Pass `Row` the columns names
* Creates a specialized `Row` class

In [38]:
Employee = Row("firstName", "lastName", "email", "salary")
Employee

<Row('firstName', 'lastName', 'email', 'salary')>

## Creating a `Employee` instance

* Pass the data to `Employee` to make a row
* Order matters ... use the same order as names

In [39]:
Employee = Row("firstName", "lastName", "email", "salary")
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee1

Row(firstName='michael', lastName='armbrust', email='no-reply@berkeley.edu', salary=100000)

## Unpacking a data list

* Suppose the data is in a list/tuple.
* Use sequence unpacking with `*`

In [40]:
empl2_info = ('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)
empl2_info

('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)

In [41]:
employee2 = Employee(*empl2_info)
employee2

Row(firstName='xiangrui', lastName='meng', email='no-reply@stanford.edu', salary=120000)

## Unpacking 

In [42]:
# Create the Employees
Employee = Row("firstName", "lastName", "email", "salary")
employees = [('michael', 'armbrust', 'no-reply@berkeley.edu', 100000),
             ('xiangrui', 'meng', 'no-reply@stanford.edu', 120000),
             ('matei', None, 'no-reply@waterloo.edu', 140000),
             (None, 'wendell', 'no-reply@berkeley.edu', 160000)]
emp_rows = [Employee(*r) for r in employees]
emp_rows

[Row(firstName='michael', lastName='armbrust', email='no-reply@berkeley.edu', salary=100000),
 Row(firstName='xiangrui', lastName='meng', email='no-reply@stanford.edu', salary=120000),
 Row(firstName='matei', lastName=None, email='no-reply@waterloo.edu', salary=140000),
 Row(firstName=None, lastName='wendell', email='no-reply@berkeley.edu', salary=160000)]